In [1]:
#basic package
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import math
import sys
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
import config_origins as config

# Define parameters

In [3]:
print('change the configuration file if not done yet!')
path_extracted_data = config.path_extracted_data
path_dataoutput = config.path_dataoutput
path_initial_data = config.path_initial_data
id_run = config.id_run
dico_HAID_date = config.dico_HAID_date
path_extracted_HA = config.path_extracted_HA

change the configuration file if not done yet!


# Download variables

In [4]:
df_HA = pd.read_csv(os.path.join(path_extracted_data,'df_REA.csv'), sep=';')
print(df_HA.shape)
df_HA.head(3)

(604, 41)


,Person_toe,Time,PenID,FocalLegringName,BirdType,CombPecks,Toes,RFPododermatitis,RFBumblefoot,RFinjuries,...,IsNonFocal,Injuries,Pododermatitis,Pododermatitis_atleastonehigher100,Bumblefoot,HasBumblefoot,Toes_inflammed,GeneralToe,Feathers,HenID
0,C,10:07,pen4,17orange,normal,75.0,NaN,0.0,0.0,0.0,...,False,0.0,0.0,0,0.0,False,False,False,NaN,hen_48
1,C,10:45,pen4,17orange,normal,100.0,NaN,0.0,0.0,0.0,...,False,0.0,0.0,0,0.0,False,False,False,NaN,hen_48
2,M,10:04,pen4,17orange,normal,25.0,NaN,0.0,0.0,0.0,...,False,0.0,0.0,0,0.0,False,False,False,NaN,hen_48


In [5]:
print(list(df_HA.columns))

['Person_toe', 'Time', 'PenID', 'FocalLegringName', 'BirdType', 'CombPecks', 'Toes', 'RFPododermatitis', 'RFBumblefoot', 'RFinjuries', 'LFPododermatitis', 'LFBumblefoot', 'LFinjuries', 'Reliability_MC', 'PassageID_toe', 'Diarrhea', 'Commentaires', 'SomeHenID', 'HAID', 'date', 'Claw', 'Person_feather', 'Reliability_SS', 'PassageID_feather', 'NeckFeather', 'BreastFeather', 'CloacaFeather', 'BackFeather', 'WingFeather', 'TailFeather', 'PenMatching', 'IsNonFocal', 'Injuries', 'Pododermatitis', 'Pododermatitis_atleastonehigher100', 'Bumblefoot', 'HasBumblefoot', 'Toes_inflammed', 'GeneralToe', 'Feathers', 'HenID']


In [6]:
#initialize set of variables
#keep LeftFoot and RightFoot separate due to slightly different conditions that might influence the realiability (e.g. 2 
#legrings on one side, e.g. how the students hold the birds)
li_cont_T = ['CombPecks', 'Pododermatitis', 'Injuries']
li_cat_T = ['Diarrhea','Bumblefoot', 'HasBumblefoot','GeneralToe']
li_cont_F = ['Feathers','NeckFeather', 'BreastFeather', 'CloacaFeather', 'BackFeather', 'WingFeather','TailFeather']

# Toes

In [7]:
df_HA_T = df_HA[df_HA['Reliability_MC']==1].copy()
print(df_HA_T.shape)
df_HA_T.head(3)

(600, 41)


,Person_toe,Time,PenID,FocalLegringName,BirdType,CombPecks,Toes,RFPododermatitis,RFBumblefoot,RFinjuries,...,IsNonFocal,Injuries,Pododermatitis,Pododermatitis_atleastonehigher100,Bumblefoot,HasBumblefoot,Toes_inflammed,GeneralToe,Feathers,HenID
0,C,10:07,pen4,17orange,normal,75.0,NaN,0.0,0.0,0.0,...,False,0.0,0.0,0,0.0,False,False,False,NaN,hen_48
1,C,10:45,pen4,17orange,normal,100.0,NaN,0.0,0.0,0.0,...,False,0.0,0.0,0,0.0,False,False,False,NaN,hen_48
2,M,10:04,pen4,17orange,normal,25.0,NaN,0.0,0.0,0.0,...,False,0.0,0.0,0,0.0,False,False,False,NaN,hen_48


In [8]:
#count numebr of focal birds that was a reability bird for the toes per pen and per HA
df_HA_T.groupby(['HAID','PenID'])['FocalLegringName'].agg(lambda x: len(set(x))).reset_index()

,HAID,PenID,FocalLegringName
0,HA1,pen11,4
1,HA1,pen12,4
2,HA1,pen3,4
3,HA1,pen4,3
4,HA1,pen5,4
5,HA1,pen8,4
6,HA1,pen9,4
7,HA2,pen10,4
8,HA2,pen11,4
9,HA2,pen12,4


In [9]:
########## INTER: mark vs camille
#create an ID for the reability bird based on the actual bird (pen & legring) but also the passage ID and the HAID
#and then verify each value of "ID4Rea" have one observation for the two raters of the feathers and the two raters for the toes
#unique ID for each observations based on their unique set of info
df_HA_T_inter = df_HA_T.copy()
df_HA_T_inter['ID4Rea'] = df_HA_T_inter.apply(lambda x: (x['PenID'].strip(' ')+'_'+x['HAID'].strip(' ')+'_'+\
                                             x['FocalLegringName'].strip(' ')+'_'+str(int(x['PassageID_toe']))), axis=1)
df_HA_T_inter = df_HA_T_inter[['Person_toe','ID4Rea']+li_cont_T+li_cat_T]
#correct persons name
df_HA_T_inter['Person_toe'] = df_HA_T_inter['Person_toe'].map(lambda x: x[0])
display(df_HA_T_inter['Person_toe'].value_counts())

# save for CCC in R
li_df = []
for v in li_cont_T+li_cat_T:
    df_ = df_HA_T_inter.pivot(index='ID4Rea', columns='Person_toe', values=v).reset_index()
    df_['HAvariable'] = v
    li_df.append(df_)
df_HA_T_inter_pivot = pd.concat(li_df)
print(df_HA_T_inter_pivot.shape)

#check nan: show rows with at least one nan
display(df_HA_T_inter_pivot[df_HA_T_inter_pivot.isna().any(axis=1)]) #--> in diarrhe only --> remove them, no correction to be made
print(df_HA_T_inter_pivot.shape)
df_HA_T_inter_pivot = df_HA_T_inter_pivot[~df_HA_T_inter_pivot.isna().any(axis=1)]
print(df_HA_T_inter_pivot.shape)

#save for CCC in R
df_HA_T_inter_pivot.to_csv(os.path.join(path_extracted_HA ,'df_HA_T_inter_pivot.csv'), index=False, sep=';')
display(df_HA_T_inter_pivot.head(3))
#nbr val per variables
display(df_HA_T_inter_pivot['HAvariable'].value_counts())
#identify potential issues
#df_ = df_HA_T_inter.groupby(['ID4Rea'])['Person_toe'].agg(lambda x: '_'.join(sorted(list(x)))).reset_index()
#df_[df_['Person_toe']!='C_M']

M    300
C    300
Name: Person_toe, dtype: int64

(2100, 4)


Person_toe,ID4Rea,C,M,HAvariable
17,pen10_HA4_27orange_2,NaN,1.0,Diarrhea
44,pen11_HA2_71green_1,NaN,0.0,Diarrhea
45,pen11_HA2_71green_2,NaN,0.0,Diarrhea
53,pen11_HA4_51green_2,1.0,NaN,Diarrhea
88,pen12_HA4_12orange_1,NaN,1.0,Diarrhea
89,pen12_HA4_12orange_2,NaN,0.0,Diarrhea
129,pen3_HA4_14green_2,NaN,NaN,Diarrhea
131,pen3_HA4_25green_2,NaN,1.0,Diarrhea
136,pen3_HA5_14green_1,1.0,NaN,Diarrhea
170,pen4_HA4_72orange_1,0.0,NaN,Diarrhea


(2100, 4)
(2074, 4)


Person_toe,ID4Rea,C,M,HAvariable
0,pen10_HA2_18orange_1,0.0,0.0,CombPecks
1,pen10_HA2_18orange_2,0.0,0.0,CombPecks
2,pen10_HA2_39black_1,50.0,50.0,CombPecks


Injuries          300
CombPecks         300
Bumblefoot        300
GeneralToe        300
HasBumblefoot     300
Pododermatitis    300
Diarrhea          274
Name: HAvariable, dtype: int64

In [10]:
########## INTRA: mark vs mark & camille vs camille
#verify each value of "ID4Rea" have one observation for the two raters of the feathers and the two raters for the toes
#unique ID for each observations based on their unique set of info
df_HA_T_intra = df_HA_T.copy()
df_HA_T_intra['ID4Rea'] = df_HA_T_intra.apply(lambda x: (x['PenID'].strip(' ')+'_'+x['HAID'].strip(' ')+'_'+\
                                             x['FocalLegringName'].strip(' ')), axis=1)
df_HA_T_intra = df_HA_T_intra[['Person_toe','ID4Rea', 'PassageID_toe']+li_cont_T+li_cat_T]
#correct persons name
df_HA_T_intra['Person_toe'] = df_HA_T_intra['Person_toe'].map(lambda x: x[0])
display(df_HA_T_intra['Person_toe'].value_counts())

# save for CCC in R
li_df = []
for v in li_cont_T+li_cat_T:
    df_ = df_HA_T_intra.pivot(index=['ID4Rea','Person_toe'], columns='PassageID_toe', values=v).reset_index()
    df_['HAvariable'] = v
    li_df.append(df_)
df_HA_T_intra_pivot = pd.concat(li_df)
print(df_HA_T_intra_pivot.shape)

#check nan: show rows with at least one nan
display(df_HA_T_intra_pivot[df_HA_T_intra_pivot.isna().any(axis=1)]) #--> in diarrhe only --> remove them, no correction to be made
print(df_HA_T_intra_pivot.shape)
df_HA_T_intra_pivot = df_HA_T_intra_pivot[~df_HA_T_intra_pivot.isna().any(axis=1)]
print(df_HA_T_intra_pivot.shape)

#save for CCC in R
df_HA_T_intra_pivot.to_csv(os.path.join(path_extracted_HA ,'df_HA_T_intra_pivot.csv'), index=False, sep=';')
display(df_HA_T_intra_pivot.head(3))

#nbr val per variable
display(df_HA_T_intra_pivot['HAvariable'].value_counts())

#identify potential issues
#df_ = df_HA_T_intra.groupby(['ID4Rea'])['Person_toe'].agg(lambda x: '_'.join(sorted(list(x)))).reset_index()
#df_[df_['Person_toe']!='C_M']

M    300
C    300
Name: Person_toe, dtype: int64

(2100, 5)


PassageID_toe,ID4Rea,Person_toe,1.0,2.0,HAvariable
16,pen10_HA4_27orange,C,1.0,NaN,Diarrhea
44,pen11_HA2_71green,C,NaN,NaN,Diarrhea
53,pen11_HA4_51green,M,1.0,NaN,Diarrhea
88,pen12_HA4_12orange,C,NaN,NaN,Diarrhea
128,pen3_HA4_14green,C,1.0,NaN,Diarrhea
129,pen3_HA4_14green,M,0.0,NaN,Diarrhea
130,pen3_HA4_25green,C,1.0,NaN,Diarrhea
137,pen3_HA5_14green,M,NaN,1.0,Diarrhea
171,pen4_HA4_72orange,M,NaN,0.0,Diarrhea
215,pen5_HA5_35grey,M,NaN,1.0,Diarrhea


(2100, 5)
(2076, 5)


PassageID_toe,ID4Rea,Person_toe,1.0,2.0,HAvariable
0,pen10_HA2_18orange,C,0.0,0.0,CombPecks
1,pen10_HA2_18orange,M,0.0,0.0,CombPecks
2,pen10_HA2_39black,C,50.0,40.0,CombPecks


Injuries          300
CombPecks         300
Bumblefoot        300
GeneralToe        300
HasBumblefoot     300
Pododermatitis    300
Diarrhea          276
Name: HAvariable, dtype: int64

# Feathers

In [11]:
df_HA_F = df_HA[(df_HA['Reliability_SS']==1)&(~df_HA['Person_feather'].isnull())&(~df_HA['PassageID_feather'].isnull())].copy()
print(df_HA_F.shape)
df_HA_F.head(3)

(471, 41)


,Person_toe,Time,PenID,FocalLegringName,BirdType,CombPecks,Toes,RFPododermatitis,RFBumblefoot,RFinjuries,...,IsNonFocal,Injuries,Pododermatitis,Pododermatitis_atleastonehigher100,Bumblefoot,HasBumblefoot,Toes_inflammed,GeneralToe,Feathers,HenID
108,C,08:20,pen3,1green,normal,25.0,NaN,0.0,0.0,50.0,...,False,50.0,0.0,0,0.0,False,False,False,98.333333,hen_81
109,C,08:20,pen3,30grey,normal,25.0,NaN,0.0,0.0,50.0,...,False,50.0,0.0,0,0.0,False,False,False,90.333333,hen_13
110,C,08:20,pen3,36green,normal,140.0,NaN,50.0,0.0,0.0,...,False,0.0,100.0,0,0.0,False,False,False,91.333333,hen_102


In [12]:
########## INTER: sabine vs Satar
#verify each value of "ID4Rea" have one observation for the two raters of the feathers and the two raters for the feathers
#unique ID for each observations based on their unique set of info
df_HA_F_inter = df_HA_F.copy()
df_HA_F_inter['ID4Rea'] = df_HA_F_inter.apply(lambda x: (x['PenID'].strip(' ')+'_'+x['HAID'].strip(' ')+'_'+\
                                             x['FocalLegringName'].strip(' ')+'_'+str(int(x['PassageID_feather']))), axis=1)
df_HA_F_inter = df_HA_F_inter[['Person_feather','ID4Rea']+li_cont_F]
#correct persons name
df_HA_F_inter['Person_feather'] = df_HA_F_inter['Person_feather'].map(lambda x: x.lower())
display(df_HA_F_inter['Person_feather'].value_counts())
#identify potential issues
#df_ = df_HA_F_inter.groupby(['ID4Rea'])['Person_feather'].agg(lambda x: '_'.join(sorted(list(x)))).reset_index()
#df_[df_['Person_feather']!='sabine_satar']

# save for CCC in R
li_df = []
for v in li_cont_F:
    df_ = df_HA_F_inter.pivot(index='ID4Rea', columns='Person_feather', values=v).reset_index()
    df_['HAvariable'] = v
    li_df.append(df_)
df_HA_F_inter_pivot = pd.concat(li_df)
print(df_HA_F_inter_pivot.shape)

#check nan: show rows with at least one nan
display(df_HA_F_inter_pivot[df_HA_F_inter_pivot.isna().any(axis=1)]) #--> Sabine missed a bird --> remove them, no correction to be made
print(df_HA_F_inter_pivot.shape)
df_HA_F_inter_pivot = df_HA_F_inter_pivot[~df_HA_F_inter_pivot.isna().any(axis=1)]
print(df_HA_F_inter_pivot.shape)

#nbr val per variables
display(df_HA_F_inter_pivot['HAvariable'].value_counts())

#save for CCC in R
df_HA_F_inter_pivot.to_csv(os.path.join(path_extracted_HA ,'df_HA_F_inter_pivot.csv'), index=False, sep=';')
display(df_HA_F_inter_pivot.head(3))

satar     236
sabine    235
Name: Person_feather, dtype: int64

(1652, 4)


Person_feather,ID4Rea,sabine,satar,HAvariable
228,pen9_HA5_13green_1,NaN,47.0,Feathers
228,pen9_HA5_13green_1,NaN,50.0,NeckFeather
228,pen9_HA5_13green_1,NaN,40.0,BreastFeather
228,pen9_HA5_13green_1,NaN,98.0,CloacaFeather
228,pen9_HA5_13green_1,NaN,99.0,BackFeather
228,pen9_HA5_13green_1,NaN,65.0,WingFeather
228,pen9_HA5_13green_1,NaN,51.0,TailFeather


(1652, 4)
(1645, 4)


Feathers         235
CloacaFeather    235
BreastFeather    235
BackFeather      235
TailFeather      235
NeckFeather      235
WingFeather      235
Name: HAvariable, dtype: int64

Person_feather,ID4Rea,sabine,satar,HAvariable
0,pen10_HA2_18orange_1,100.000000,99.333333,Feathers
1,pen10_HA2_18orange_2,100.000000,100.000000,Feathers
2,pen10_HA2_39black_1,90.666667,100.000000,Feathers


In [22]:
#nbr of observations for the feathers: in total
df_HA_F_inter_pivot.groupby(['HAvariable']).count().reset_index()

Person_feather,HAvariable,ID4Rea,sabine,satar
0,BackFeather,235,235,235
1,BreastFeather,235,235,235
2,CloacaFeather,235,235,235
3,Feathers,235,235,235
4,NeckFeather,235,235,235
5,TailFeather,235,235,235
6,WingFeather,235,235,235


In [15]:
########## INTRA: sabine vs Sabine / Satar vs Satar
#verify each value of "ID4Rea" have one observation for the two raters of the feathers and the two raters for the feathers
#unique ID for each observations based on their unique set of info
df_HA_F_intra = df_HA_F.copy()
df_HA_F_intra['ID4Rea'] = df_HA_F_intra.apply(lambda x: (x['PenID'].strip(' ')+'_'+x['HAID'].strip(' ')+'_'+\
                                             x['FocalLegringName'].strip(' ')), axis=1)
df_HA_F_intra = df_HA_F_intra[['Person_feather','ID4Rea','HAID','PassageID_feather']+li_cont_F]
#correct persons name
df_HA_F_intra['Person_feather'] = df_HA_F_intra['Person_feather'].map(lambda x: x.lower())
display(df_HA_F_intra['Person_feather'].value_counts())
################### save for CCC in R
li_df = []
for v in li_cont_F:    
    df_ = df_HA_F_intra.pivot(index=['ID4Rea','HAID','Person_feather'], columns='PassageID_feather', values=v).reset_index()
    df_['HAvariable'] = v
    li_df.append(df_)
df_HA_F_intra_pivot = pd.concat(li_df)
print(df_HA_F_intra_pivot.shape)

#check nan: show rows with at least one nan
display(df_HA_F_intra_pivot[df_HA_F_intra_pivot.isna().any(axis=1)]) #--> in diarrhe only --> remove them, no correction to be made
print(df_HA_F_intra_pivot.shape)
df_HA_F_intra_pivot = df_HA_F_intra_pivot[~df_HA_F_intra_pivot.isna().any(axis=1)]
print(df_HA_F_intra_pivot.shape)

#nbr val per variable
display(df_HA_F_intra_pivot['HAvariable'].value_counts())

#save for CCC in R
df_HA_F_intra_pivot.to_csv(os.path.join(path_extracted_HA ,'df_HA_F_intra_pivot.csv'), index=False, sep=';')
display(df_HA_F_intra_pivot.head(3))

satar     236
sabine    235
Name: Person_feather, dtype: int64

(1652, 6)


PassageID_feather,ID4Rea,HAID,Person_feather,1.0,2.0,HAvariable
228,pen9_HA5_13green,HA5,sabine,NaN,63.666667,Feathers
228,pen9_HA5_13green,HA5,sabine,NaN,71.000000,NeckFeather
228,pen9_HA5_13green,HA5,sabine,NaN,57.000000,BreastFeather
228,pen9_HA5_13green,HA5,sabine,NaN,100.000000,CloacaFeather
228,pen9_HA5_13green,HA5,sabine,NaN,95.000000,BackFeather
228,pen9_HA5_13green,HA5,sabine,NaN,65.000000,WingFeather
228,pen9_HA5_13green,HA5,sabine,NaN,63.000000,TailFeather


(1652, 6)
(1645, 6)


Feathers         235
CloacaFeather    235
BreastFeather    235
BackFeather      235
TailFeather      235
NeckFeather      235
WingFeather      235
Name: HAvariable, dtype: int64

PassageID_feather,ID4Rea,HAID,Person_feather,1.0,2.0,HAvariable
0,pen10_HA2_18orange,HA2,sabine,100.000000,100.000000,Feathers
1,pen10_HA2_18orange,HA2,satar,99.333333,100.000000,Feathers
2,pen10_HA2_39black,HA2,sabine,90.666667,96.333333,Feathers


In [17]:
#nbr of observations for the feathers: in total
df_HA_F_intra_pivot.groupby(['Person_feather','HAvariable']).count().reset_index()

PassageID_feather,Person_feather,HAvariable,ID4Rea,HAID,1.0,2.0
0,sabine,BackFeather,117,117,117,117
1,sabine,BreastFeather,117,117,117,117
2,sabine,CloacaFeather,117,117,117,117
3,sabine,Feathers,117,117,117,117
4,sabine,NeckFeather,117,117,117,117
5,sabine,TailFeather,117,117,117,117
6,sabine,WingFeather,117,117,117,117
7,satar,BackFeather,118,118,118,118
8,satar,BreastFeather,118,118,118,118
9,satar,CloacaFeather,118,118,118,118


In [20]:
#nbr of observations for the feathers: per timestamps
df_HA_F_intra_pivot.groupby(['Person_feather','HAvariable','HAID']).count().reset_index()

PassageID_feather,Person_feather,HAvariable,HAID,ID4Rea,1.0,2.0
0,sabine,BackFeather,HA2,29,29,29
1,sabine,BackFeather,HA3,26,26,26
2,sabine,BackFeather,HA4,32,32,32
3,sabine,BackFeather,HA5,30,30,30
4,sabine,BreastFeather,HA2,29,29,29
5,sabine,BreastFeather,HA3,26,26,26
6,sabine,BreastFeather,HA4,32,32,32
7,sabine,BreastFeather,HA5,30,30,30
8,sabine,CloacaFeather,HA2,29,29,29
9,sabine,CloacaFeather,HA3,26,26,26
